 “The author(s) acknowledges the use of Github copilot in the preparation or completion of this assignment. The Github copilot was used in the following way(s) in this assignment: aiding in codework, and clean-up of code.”

In [1]:
# reading in the libraries and functions that we will need as we do this work.
%pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
import scipy.stats as st
import statsmodels.api as sm 
import pylab as py 

# here are some of the tools we will use for our analyses
from sklearn.linear_model import LinearRegression
from sklearn.metrics import PredictionErrorDisplay
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

Note: you may need to restart the kernel to use updated packages.


In [2]:
print("Testing the push operation.......")

Testing the push operation.......


In [9]:
# read in the data to dataframe called ames
ames2 = pd.read_csv("https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/ames2.csv", na_values=['?'])
# replace the ? in the data with NaN for missing values
ames2 = ames2.replace([' ?'],np.nan)
# show information about the dataframe
ames2.info()

ames2.dropna(inplace=True)
ames2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotArea       1000 non-null   int64  
 1   GrLivArea     1000 non-null   int64  
 2   OverallQual   1000 non-null   int64  
 3   FullBath      1000 non-null   int64  
 4   TotalPorchSF  1000 non-null   int64  
 5   BsmtFinSF1    1000 non-null   int64  
 6   GaragedCat    1000 non-null   int64  
 7   TotRmsAbvGrd  1000 non-null   int64  
 8   TotalBsmtSF   1000 non-null   int64  
 9   YearBuilt     1000 non-null   int64  
 10  YrSold        1000 non-null   int64  
 11  BsmtUnfSF     1000 non-null   int64  
 12  GarageArea    1000 non-null   int64  
 13  MoSold        1000 non-null   int64  
 14  PavedDrive01  1000 non-null   int64  
 15  MSSubClass    1000 non-null   int64  
 16  logSalePrice  1000 non-null   float64
dtypes: float64(1), int64(16)
memory usage: 132.9 KB


,LotArea,GrLivArea,OverallQual,FullBath,TotalPorchSF,BsmtFinSF1,GaragedCat,TotRmsAbvGrd,TotalBsmtSF,YearBuilt,YrSold,BsmtUnfSF,GarageArea,MoSold,PavedDrive01,MSSubClass,logSalePrice
0,9120,1820,7,2,100,329,1,8,1026,1925,2008,697,240,6,0,50,12.122691
1,4060,1337,6,2,68,266,0,5,1405,1998,2008,1139,511,8,1,120,12.106252
2,34650,1056,5,1,0,1056,0,5,1056,1955,2006,0,572,1,1,190,11.884489
3,21750,1771,5,1,0,0,0,9,0,1960,2009,0,336,11,1,20,11.652687
4,11500,845,4,1,0,0,1,5,0,1957,2009,0,290,1,0,20,11.338572


In [ ]:
#identify the target variable
numeric_cols = ames2.select_dtypes(include=[np.number]).columns

target = None
for col in ames2.columns:
    if 'price' in col.lower() or 'sale' in col.lower():
        target = col
        print(f"Target variable identified: {target}")
        break
#AI was used for this section of code below
if target is None:
    # If no price column found, show columns and ask to identify
    print("\nNo price-related column found automatically.")
    print("Available numeric columns for target consideration:")
    print(numeric_cols)
    target = 'SalePrice'  # Default assumption
    if target in ames2.columns:
        print(f"\nUsing {target} as target variable")
#Ai was not used for this section of code below but refrenced online videos
# Correlation analysis with target
print(f"\n{'='*60}")
print(f"CORRELATION WITH TARGET ({target})")
print(f"{'='*60}")
if target in ames2.columns:
    correlations = ames2[numeric_cols].corr()[target].sort_values(ascending=False)
    print("\nTop 15 correlations with target:")
    print(correlations.head(15))
else:
    print(f"\nTarget variable '{target}' not found in dataset")
    correlations = None

Target variable identified: logSalePrice

CORRELATION WITH TARGET (logSalePrice)

Top 15 correlations with target:
logSalePrice    1.000000
OverallQual     0.819959
GrLivArea       0.729548
GarageArea      0.667635
TotalBsmtSF     0.639556
FullBath        0.616202
YearBuilt       0.595846
TotRmsAbvGrd    0.556956
BsmtFinSF1      0.379405
PavedDrive01    0.315813
LotArea         0.298528
BsmtUnfSF       0.226391
TotalPorchSF    0.190537
MoSold          0.074630
YrSold         -0.009859
Name: logSalePrice, dtype: float64


In [ ]:
def calculate_rmsq_adj(y_true, y_pred, n_predictors):
    """Calculate R-squared and adjusted R-squared"""
    ss_res = np.sum((y_true - y_pred) ** 2)  # Residual sum of squares
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # Total sum of squares
    r_squared = 1 - (ss_res / ss_tot)
    n = len(y_true)
    r_squared_adj = 1 - (1 - r_squared) * (n - 1) / (n - n_predictors - 1)
    rmse = np.sqrt(ss_res / n)
    return r_squared, r_squared_adj, rmse

# Prepare data - separate features and target
y = ames2['logSalePrice']
X_all_features = ames2.drop('logSalePrice', axis=1)

print(f"\n{'='*60}")
print("BUILDING AND COMPARING REGRESSION MODELS")
print(f"{'='*60}")

# Store results for comparison
model_results = {}
#AI was used for this section of code below with refrence to online tutorial videos
# Model 1: Using Top 3 predictors from data above
print("\nModel 1: Top 3 Predictors (OverallQual, GrLivArea, GarageArea)")
features_1 = ['OverallQual', 'GrLivArea', 'GarageArea']
X1 = sm.add_constant(ames2[features_1])
model_1 = sm.OLS(y, X1).fit()
r2_1, r2adj_1, rmse_1 = calculate_rmsq_adj(y, model_1.fittedvalues, len(features_1))
model_results['Model 1 (Top 3)'] = {
    'r2': r2_1, 'r2_adj': r2adj_1, 'rmse': rmse_1, 'n_vars': len(features_1), 'model': model_1
}
print(f"  R² = {r2_1:.6f}, R²_adj = {r2adj_1:.6f}, RMSE = {rmse_1:.6f}")
#Ai was not used past this point but refrenced AI code above and online tutorial videos
# Model 2: Top 5 predictors using data above
print("\nModel 2: Top 5 Predictors (OverallQual, GrLivArea, GarageArea, TotalBsmtSF, FullBath)")
features_2 = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', 'FullBath']
X2 = sm.add_constant(ames2[features_2])
model_2 = sm.OLS(y, X2).fit()
r2_2, r2adj_2, rmse_2 = calculate_rmsq_adj(y, model_2.fittedvalues, len(features_2))
model_results['Model 2 (Top 5)'] = {
    'r2': r2_2, 'r2_adj': r2adj_2, 'rmse': rmse_2, 'n_vars': len(features_2), 'model': model_2
}
print(f"  R² = {r2_2:.6f}, R²_adj = {r2adj_2:.6f}, RMSE = {rmse_2:.6f}")

# Model 3: Top 7 predictors using data above
print("\nModel 3: Top 7 Predictors")
features_3 = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'TotRmsAbvGrd']
X3 = sm.add_constant(ames2[features_3])
model_3 = sm.OLS(y, X3).fit()
r2_3, r2adj_3, rmse_3 = calculate_rmsq_adj(y, model_3.fittedvalues, len(features_3))
model_results['Model 3 (Top 7)'] = {
    'r2': r2_3, 'r2_adj': r2adj_3, 'rmse': rmse_3, 'n_vars': len(features_3), 'model': model_3
}
print(f"  R² = {r2_3:.6f}, R²_adj = {r2adj_3:.6f}, RMSE = {rmse_3:.6f}")

# Model 5: Include all numeric features
print("\nModel 5: All Available Features")
X5 = sm.add_constant(X_all_features)
model_5 = sm.OLS(y, X5).fit()
r2_5, r2adj_5, rmse_5 = calculate_rmsq_adj(y, model_5.fittedvalues, X_all_features.shape[1])
model_results['Model 5 (All Features)'] = {
    'r2': r2_5, 'r2_adj': r2adj_5, 'rmse': rmse_5, 'n_vars': X_all_features.shape[1], 'model': model_5
}
print(f"  R² = {r2_5:.6f}, R²_adj = {r2adj_5:.6f}, RMSE = {rmse_5:.6f}")



BUILDING AND COMPARING REGRESSION MODELS

Model 1: Top 3 Predictors (OverallQual, GrLivArea, GarageArea)
  R² = 0.796108, R²_adj = 0.795494, RMSE = 0.182525

Model 2: Top 5 Predictors (OverallQual, GrLivArea, GarageArea, TotalBsmtSF, FullBath)
  R² = 0.822932, R²_adj = 0.822041, RMSE = 0.170096

Model 3: Top 7 Predictors
  R² = 0.845183, R²_adj = 0.844091, RMSE = 0.159049

Model 5: All Available Features
  R² = 0.868494, R²_adj = 0.866354, RMSE = 0.146587


In [ ]:
#best model summary using R**2 adjusted and display results
print(f"\n{'='*60}")
print("FINAL BEST MODEL: MODEL 5 (ALL FEATURES)")
print(f"{'='*60}")

best_model = model_results['Model 5 (All Features)']['model']

print("\nComplete Model Summary:")
print(best_model.summary())


FINAL BEST MODEL: MODEL 5 (ALL FEATURES)

Complete Model Summary:
                            OLS Regression Results                            
Dep. Variable:           logSalePrice   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                     405.7
Date:                Tue, 10 Feb 2026   Prob (F-statistic):               0.00
Time:                        15:51:23   Log-Likelihood:                 501.20
No. Observations:                1000   AIC:                            -968.4
Df Residuals:                     983   BIC:                            -885.0
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------